In [1]:
# ---Script that finds flares in TESS LCs---
#
# last update: 29.04.2019
#
# Run as:
# $ python3 script.py *lc.fits
# in the folder with all the light curves
#
# Note:
# TIC 77041172 failed with:
# ValueError: cannot reshape array of size 2198 into shape (19737,)
#
# Column description
# tstart, tstop      --- BJD-2457000 start and end time of flare detection
# cstart, cstop      --- start and end cadence number of flare detection
# istart, istop      --- start and end index into array of flare detection
# ampl_rec           --- recoverd flare amplitude relative to quiescent flux
# ed_rec, ed_rec_err --- recovered ED and uncertainty in seconds
# TIC                --- TESS ID
# Campaign           --- TESS mission campaign

import numpy as np
import pandas as pd
from astropy.io import fits
from altaipony.flarelc import FlareLightCurve
import sys

import matplotlib.pyplot as plt
%matplotlib inline

def write_flares_to_file(flc, cluster):
    '''
    Write the resulting flare table to file, adding
    it  to the rest.
    '''
    cols = ['TIC', 'ampl_rec', 'cstart', 'cstop', 
            'ed_rec', 'ed_rec_err', 'istart', 'istop', 
            'tstart', 'tstop', 'Campaign']
            #'saturation_f10']
    try:
        df = pd.read_csv('{0}_flares.csv'.format(cluster),
            usecols=cols)
    except:
        print('Create a file named {}_flares.csv'.format(cluster))
        df = pd.DataFrame(columns=cols)
        
    lbefore = flc.flares[~flc.flares.ed_rec.isnull()].shape[0]
    if lbefore == 0:
        emptydf = pd.DataFrame(dict(zip(cols, [flc.targetid] + [np.nan] * (len(cols) - 1))), index=[0])
        flc.flares = flc.flares.append(emptydf, ignore_index=True)
        print('Added an empty row for TIC {}'.format(flc.targetid))
    flc.flares['TIC'] = flc.targetid
    flc.flares['Campaign'] = flc.campaign
    #flc.flares = flc.get_saturation(return_level=True) maybe later
    print(flc.flares)
    df = df.append(flc.flares, ignore_index=True)
    lafter = df[~df.ed_rec.isnull()].shape[0]
   # print('Added {} flares to the {} found so far in {}.'.format(lafter-lbefore, lbefore, cluster))
    df.to_csv('{0}_flares.csv'.format(cluster), index=False)
    return

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib64/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /work1/eilin/Flares_in_Clusters_2/RESULTS_Github/flares-in-clusters-with-k2-ii/flaresinclustersii/lib64/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
/usr/lib64/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /work1/eilin/Flares_in_Clusters_2/RESULTS_Github/flares-in-clusters-with-k2-ii/flaresinclustersii/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.

In [2]:
paths = pd.read_csv("/work1/eilin/TESS_UCDs/TESS_UCD_flares/custom_aperture/lclist.txt", header=None)
paths = list(paths[0].values)
len(paths)

191

In [ ]:
for i, path in enumerate(paths):
    print(path, "\n")
    path = "/work1/eilin/TESS_UCDs/TESS_UCD_flares/custom_aperture/" + path
    cluster = "/work1/eilin/TESS_UCDs/flare_tables/2019_10_25_TESSUCDs"
    hdu = fits.open(path)
    data = hdu[1].data

    sector = int(path.split("-")[1][-2:])
    TIC = int(path.split("-")[2])

    print("\nStarted TIC {} ({}), LC nr. {}\n------------------------------\n".format(TIC, sector, i))

    flc = FlareLightCurve(time=data["TIME"],
                        flux=data["FLUX"],
                        flux_err=data["FLUX_ERR"],
                        quality=data["QUALITY"],
                        cadenceno=data["CADENCENO"],
                        targetid=TIC,
                        campaign=sector)

    if TIC in [29928567, 142052876, 293561794, 471012740, 142052876]: #rapid rotators
        w = 5
    elif TIC in [98874143, 272349442, 388903843, 332623751, 328254412, 44892011, 44892011, 117733581, 29780677, 340703996]: #intermediate rotators/ variability time scale / some weird systematics
        w = 25
    elif TIC in [471012740]:
        w = 555
    else:
        w = 25
    flcd = flc.detrend("savgol", window_length=w)
    flcd = flcd.find_flares()
   
    write_flares_to_file(flcd, cluster)
    hdu.close()
    
    print("\nFinished TIC {} ({})\n------------------------------\n".format(TIC, sector))


0ca_tess2019082014090-s0001-0000000029780677-0000-s_lc.fits 


Started TIC 29780677 (1), LC nr. 0
------------------------------



Found 0 candidate(s) in the (0,9223) gap.
Found 2 candidate(s) in the (9223,15240) gap.
Found 0 candidate(s) in the (15240,18104) gap.


Create a file named /work1/eilin/TESS_UCDs/flare_tables/2019_10_25_TESSUCDs_flares.csv
   ampl_rec cstart  cstop       ed_rec  ed_rec_err istart  istop  \
0  2.421832  81083  81126  1621.434404    4.646887   9518   9561   
1  0.179638  83396  83416   231.591749    4.078065  11803  11823   

   total_n_valid_data_points       tstart        tstop       TIC  Campaign  
0                    18104.0  1340.071717  1340.131439  29780677         1  
1                    18104.0  1343.284226  1343.312004  29780677         1  

Finished TIC 29780677 (1)
------------------------------

1ca_tess2019082014114-s0001-0000000142052876-0000-s_lc.fits 


Started TIC 142052876 (1), LC nr. 1
------------------------------



Found 6 candidate(s) in the (0,9223) gap.
Found 0 candidate(s) in the (9223,15240) gap.
Found 2 candidate(s) in the (15240,18104) gap.


   ampl_rec cstart  cstop      ed_rec  ed_rec_err istart  istop  \
0  0.015014  74161  74165    5.772812    0.408417   3579   3583   
1  0.022008  76823  76831   12.725052    0.549814   6163   6171   
2  0.015628  77660  77684   37.270688    1.942071   6967   6975   
3  0.090806  77718  77733   42.499317    0.607565   7009   7023   
4  0.015222  78410  78413    4.381121    0.335384   7697   7700   
5  0.010119  79157  79164    7.784945    0.532556   8437   8444   
6  0.448444  90441  90472  193.794515    0.653799  18025  18056   
7  0.014571  90482  90485    3.772694    0.339027  18066  18069   

   total_n_valid_data_points       tstart        tstop        TIC  Campaign  
0                    18104.0  1330.457578  1330.463133  142052876         1  
1                    18104.0  1334.154755  1334.165866  142052876         1  
2                    18104.0  1335.317236  1335.350569  142052876         1  
3                    18104.0  1335.397790  1335.418623  142052876         1  
4     

...

      ampl_rec  cstart   cstop        ed_rec  ed_rec_err istart istop  \
0  1623.933716  208723  208730  1.099236e+06         0.0      1     7   

   total_n_valid_data_points       tstart        tstop     TIC  Campaign  
0                    13427.0  1517.353356  1517.363078  926898         8  

Finished TIC 926898 (8)
------------------------------

104ca_tess2019090911104-s0008-0000000007975441-0000-s_lc.fits 


Started TIC 7975441 (8), LC nr. 104
------------------------------



Found 2 candidate(s) in the (0,8387) gap.
Found 1 candidate(s) in the (8387,12958) gap.


   ampl_rec  cstart   cstop      ed_rec  ed_rec_err istart  istop  \
0  0.195106  213756  213760   52.193400    3.896217   4990   4994   
1  0.357090  216170  216181  252.161943    6.365283   7397   7408   
2  0.306365  223541  223544   69.099931    3.413096  10035  10038   

   total_n_valid_data_points       tstart        tstop      TIC  Campaign  
0                    12958.0  1524.344099  1524.349655  7975441         8  
1                    12958.0  1527.696846  1527.712123  7975441         8  
2                    12958.0  1537.934154  1537.938320  7975441         8  

Finished TIC 7975441 (8)
------------------------------

105ca_tess2019090911241-s0008-0000000011654435-0000-s_lc.fits 


Started TIC 11654435 (8), LC nr. 105
------------------------------



Found 1 candidate(s) in the (0,8406) gap.
Found 8 candidate(s) in the (8406,13427) gap.


    ampl_rec  cstart   cstop        ed_rec  ed_rec_err istart istop  \
0  22.828621  208723  208759  16959.358027    0.000000      1    33   
1   0.417624  221436  222559  15777.244220    7.663637   8407  9522   
2   0.030650  222565  222617     96.081800    2.022453   9528  9580   
3   0.016386  222624  222653     45.085689    1.562167   9587  9616   
4   0.016974  222659  222663      6.618566    0.576409   9622  9626   
5   0.015967  222690  222694      6.266548    0.576141   9653  9657   
6   0.013809  222699  222702      4.371808    0.505698   9662  9665   
7   0.014431  222708  222716     10.358207    0.822855   9671  9679   
8   0.014473  222737  222740      4.423220    0.503633   9700  9703   

   total_n_valid_data_points       tstart        tstop       TIC  Campaign  
0                    13427.0  1517.352037  1517.402037  11654435         8  
1                    13427.0  1535.009042  1536.568754  11654435         8  
2                    13427.0  1536.577088  1536.649309  11

In [4]:
df = pd.read_csv("/work1/eilin/TESS_UCDs/flare_tables/2019_10_25_TESSUCDs_flares.csv")

180724005 (3) - flares are not real because light curve makes an unexpected excursion

In [5]:
df.head()

,Campaign,TIC,ampl_rec,cstart,cstop,ed_rec,ed_rec_err,istart,istop,total_n_valid_data_points,tstart,tstop
0,1,29780677,2.424056,81083.0,81127.0,1638.774308,4.680147,9518.0,9562.0,NaN,1340.071717,1340.132828
1,1,29780677,0.185100,83396.0,83416.0,244.729747,4.116730,11803.0,11823.0,NaN,1343.284226,1343.312004
2,1,142052876,0.015014,74161.0,74165.0,5.772812,0.408417,3579.0,3583.0,NaN,1330.457578,1330.463133
3,1,142052876,0.022008,76823.0,76831.0,12.725052,0.549814,6163.0,6171.0,NaN,1334.154755,1334.165866
4,1,142052876,0.015628,77660.0,77684.0,37.270688,1.942071,6967.0,6975.0,NaN,1335.317236,1335.350569


In [6]:
plt.figure(figsize=(16,5))
for c, dfs in df.groupby("Campaign"):
    plt.scatter(dfs.tstart, dfs.ampl_rec, label=c)
plt.legend()
plt.yscale("log")
plt.

SyntaxError: invalid syntax (<ipython-input-6-33a19e34a802>, line 6)

In [ ]:
df[df.Campaign ==10].groupby("TIC").count()